Modelo Random Forest con Preprocesado

Entrena un modelo Random Forest con RandomizedSearchCV y genera `submission_rf.csv`.

In [4]:
# Importar librerías necesarias
from google.colab import files
import zipfile
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import os
import time

# Configurar Kaggle API y descargar datos
print("Por favor, sube el archivo kaggle.json")
uploaded = files.upload()

if 'kaggle.json' in uploaded:
    !mkdir -p ~/.kaggle
    !mv kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
    print("kaggle.json configurado correctamente")
else:
    raise FileNotFoundError("No se subió kaggle.json")

!pip install kaggle --quiet
!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia
if os.path.exists("udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip"):
    with zipfile.ZipFile("udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip", "r") as zip_ref:
        zip_ref.extractall("datos")
    print("Datos extraídos en datos/")
    if os.path.exists('datos/train.csv'):
        print(f"train.csv encontrado, tamaño: {os.path.getsize('datos/train.csv')} bytes")
    if os.path.exists('datos/test.csv'):
        print(f"test.csv encontrado, tamaño: {os.path.getsize('datos/test.csv')} bytes")
    else:
        raise FileNotFoundError("test.csv no se encuentra en datos/")
else:
    raise FileNotFoundError("Fallo al descargar el ZIP")

# Cargar datos
try:
    train = pd.read_csv('datos/train.csv', engine='python', encoding='utf-8')
    test = pd.read_csv('datos/test.csv', engine='python', encoding='utf-8')
    print("Archivos cargados correctamente")
    test_ids = test['ID']
except Exception as e:
    print(f"Error al cargar archivos: {e}")
    raise

# Eliminar columna duplicada si existe
if 'FAMI_TIENEINTERNET.1' in train.columns:
    print("Eliminando columna duplicada FAMI_TIENEINTERNET.1 en train")
    train = train.drop('FAMI_TIENEINTERNET.1', axis=1)
if 'FAMI_TIENEINTERNET.1' in test.columns:
    print("Eliminando columna duplicada FAMI_TIENEINTERNET.1 en test")
    test = test.drop('FAMI_TIENEINTERNET.1', axis=1)

# Estandarizar valores
train['FAMI_ESTRATOVIVIENDA'] = train['FAMI_ESTRATOVIVIENDA'].str.lower().fillna('sin estrato')
test['FAMI_ESTRATOVIVIENDA'] = test['FAMI_ESTRATOVIVIENDA'].str.lower().fillna('sin estrato')

# Manejar valores faltantes
for df in [train, test]:
    df['ESTU_VALORMATRICULAUNIVERSIDAD'] = df['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna('no pagó matrícula')
    df['ESTU_HORASSEMANATRABAJA'] = df['ESTU_HORASSEMANATRABAJA'].fillna('0')
    df['FAMI_EDUCACIONPADRE'] = df['FAMI_EDUCACIONPADRE'].fillna('No sabe')
    df['FAMI_EDUCACIONMADRE'] = df['FAMI_EDUCACIONMADRE'].fillna('No sabe')
    df['FAMI_TIENEINTERNET'] = df['FAMI_TIENEINTERNET'].fillna('No')
    df['FAMI_TIENECOMPUTADOR'] = df['FAMI_TIENECOMPUTADOR'].fillna('No')
    df['FAMI_TIENELAVADORA'] = df['FAMI_TIENELAVADORA'].fillna('No')
    df['FAMI_TIENEAUTOMOVIL'] = df['FAMI_TIENEAUTOMOVIL'].fillna('No')
    df['ESTU_PAGOMATRICULAPROPIO'] = df['ESTU_PAGOMATRICULAPROPIO'].fillna('No')
    df['ESTU_PRIVADO_LIBERTAD'] = df['ESTU_PRIVADO_LIBERTAD'].fillna('N')

# Codificación ordinal
valormatricula_map = {
    'no pagó matrícula': 0,
    'menos de 500 mil': 1,
    'entre 500 mil y menos de 1 millón': 2,
    'entre 1 millón y menos de 2.5 millones': 3,
    'entre 2.5 millones y menos de 4 millones': 4,
    'entre 4 millones y menos de 5.5 millones': 5,
    'entre 5.5 millones y menos de 7 millones': 6,
    'más de 7 millones': 7
}
train['ESTU_VALORMATRICULAUNIVERSIDAD'] = train['ESTU_VALORMATRICULAUNIVERSIDAD'].str.lower().map(valormatricula_map)
test['ESTU_VALORMATRICULAUNIVERSIDAD'] = test['ESTU_VALORMATRICULAUNIVERSIDAD'].str.lower().map(valormatricula_map)

horastrabajo_map = {
    '0': 0,
    'menos de 10 horas': 1,
    'entre 11 y 20 horas': 2,
    'entre 21 y 30 horas': 3,
    'más de 30 horas': 4
}
train['ESTU_HORASSEMANATRABAJA'] = train['ESTU_HORASSEMANATRABAJA'].str.lower().map(horastrabajo_map)
test['ESTU_HORASSEMANATRABAJA'] = test['ESTU_HORASSEMANATRABAJA'].str.lower().map(horastrabajo_map)

# Aplicar Label Encoding
label_cols = ['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE']
le_dict = {}
for col in label_cols:
    le = LabelEncoder()
    le.fit(train[col])
    train[col] = le.transform(train[col])
    test[col] = test[col].map(lambda s: '<unknown>' if s not in le.classes_ else s)
    le.classes_ = np.append(le.classes_, '<unknown>')
    test[col] = le.transform(test[col])
    le_dict[col] = le

# Aplicar One-Hot Encoding
one_hot_cols = ['FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_TIENECOMPUTADOR',
                'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PAGOMATRICULAPROPIO',
                'ESTU_PRIVADO_LIBERTAD']
train = pd.get_dummies(train, columns=one_hot_cols, drop_first=True)
test = pd.get_dummies(test, columns=one_hot_cols, drop_first=True)

# Asegurar que test tenga las mismas columnas que train
missing_cols = set(train.columns) - set(test.columns) - {'ID', 'RENDIMIENTO_GLOBAL'}
for col in missing_cols:
    test[col] = 0
test_preprocessed = test[train.drop(['ID', 'RENDIMIENTO_GLOBAL'], axis=1).columns]

# Normalizar columnas numéricas con StandardScaler
num_cols = ['coef_1', 'coef_2', 'coef_3', 'coef_4']
scaler = StandardScaler()
train[num_cols] = scaler.fit_transform(train[num_cols])
test_preprocessed[num_cols] = scaler.transform(test_preprocessed[num_cols])

# Eliminar columnas duplicadas
train = train.loc[:, ~train.columns.duplicated()]
test_preprocessed = test_preprocessed.loc[:, ~test_preprocessed.columns.duplicated()]

# Verificar datos
print('Columnas en train:', train.columns.tolist())
print('Tipos de datos en train:\n', train.dtypes)
print('Columnas en test_preprocessed:', test_preprocessed.columns.tolist())
print('Tipos de datos en test_preprocessed:\n', test_preprocessed.dtypes)
for col in train.columns:
    if train[col].dtype == 'object' and col not in ['ID', 'RENDIMIENTO_GLOBAL']:
        print(f"Columna con strings en train: {col}, valores: {train[col].unique()}")
for col in test_preprocessed.columns:
    if test_preprocessed[col].dtype == 'object':
        print(f"Columna con strings en test_preprocessed: {col}, valores: {test_preprocessed[col].unique()}")

# Separar características y variable objetivo
X_04 = train.drop(['ID', 'RENDIMIENTO_GLOBAL'], axis=1)
y_04 = train['RENDIMIENTO_GLOBAL']

# Convertir etiquetas categóricas a numéricas
label_encoder_y = LabelEncoder()
y_04 = label_encoder_y.fit_transform(y_04)

# Dividir en conjunto de entrenamiento y validación
X_train_04, X_val_04, y_train_04, y_val_04 = train_test_split(X_04, y_04, test_size=0.2, random_state=42, stratify=y_04)

print('Tamaño del conjunto de entrenamiento (04):', X_train_04.shape)
print('Tamaño del conjunto de validación (04):', X_val_04.shape)

# Entrenar modelo Random Forest con parámetros fijos
start_time = time.time()
rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
rf.fit(X_train_04, y_train_04)
print(f"Tiempo de entrenamiento de Random Forest: {(time.time() - start_time) / 60:.2f} minutos")

# Evaluar en conjunto de validación
y_pred_04 = rf.predict(X_val_04)
print('Precisión en validación (04):', accuracy_score(y_val_04, y_pred_04))
print('Reporte de clasificación (04):\n', classification_report(y_val_04, y_pred_04))

# Generar predicciones para test.csv
test_pred_04 = rf.predict(test_preprocessed)

# Convertir predicciones numéricas a etiquetas categóricas
test_pred_04 = label_encoder_y.inverse_transform(test_pred_04)

# Crear archivo de sumisión
submission_04 = pd.DataFrame({'ID': test_ids, 'RENDIMIENTO_GLOBAL': test_pred_04})
submission_04.to_csv('submission_rf.csv', index=False)
print('Archivo de sumisión guardado como submission_rf.csv')

# Verificar que el archivo se guardó
if os.path.exists('submission_rf.csv'):
    print(f"submission_rf.csv existe, tamaño: {os.path.getsize('submission_rf.csv')} bytes")
else:
    print("Error: submission_rf.csv no se guardó correctamente")

Por favor, sube el archivo kaggle.json


Saving kaggle.json to kaggle.json
kaggle.json configurado correctamente
udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)
Datos extraídos en datos/
train.csv encontrado, tamaño: 143732449 bytes
test.csv encontrado, tamaño: 59185250 bytes
Archivos cargados correctamente
Eliminando columna duplicada FAMI_TIENEINTERNET.1 en train
Eliminando columna duplicada FAMI_TIENEINTERNET.1 en test


/tmp/ipython-input-4-2629057451.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_preprocessed[num_cols] = scaler.transform(test_preprocessed[num_cols])


Columnas en train: ['ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL', 'coef_1', 'coef_2', 'coef_3', 'coef_4', 'FAMI_ESTRATOVIVIENDA_estrato 2', 'FAMI_ESTRATOVIVIENDA_estrato 3', 'FAMI_ESTRATOVIVIENDA_estrato 4', 'FAMI_ESTRATOVIVIENDA_estrato 5', 'FAMI_ESTRATOVIVIENDA_estrato 6', 'FAMI_ESTRATOVIVIENDA_sin estrato', 'FAMI_TIENEINTERNET_Si', 'FAMI_TIENECOMPUTADOR_Si', 'FAMI_TIENELAVADORA_Si', 'FAMI_TIENEAUTOMOVIL_Si', 'ESTU_PAGOMATRICULAPROPIO_Si', 'ESTU_PRIVADO_LIBERTAD_S']
Tipos de datos en train:
 ID                                    int64
PERIODO                               int64
ESTU_PRGM_ACADEMICO                   int64
ESTU_PRGM_DEPARTAMENTO                int64
ESTU_VALORMATRICULAUNIVERSIDAD        int64
ESTU_HORASSEMANATRABAJA               int64
FAMI_EDUCACIONPADRE                   int64
FAMI_EDUCACIONMADRE                   int64


# Conclusiones
- El preprocesamiento  utiliza codificación ordinal, Label Encoding, One-Hot Encoding y StandardScaler.
- El modelo Random Forest fue optimizado con RandomizedSearchCV.
- La precisión en validación indica el rendimiento esperado.
- El archivo `submission_rf.csv` está listo para Kaggle.